# Trabalho Prático de Teoria de Códigos 1

### Hugo Rocha, PG52250, MMC 
### Simão Quintela, PG52257, MMC

**Enunciado:**


**Construa um código linear binário [7, 3, d], com d não inferior a 3.**
**Codifique uma mensagem e cause erros que sejam corrigíveis. Usando os síndromes, corrija a mensagem.**

##### Construção da matriz geradora do código sobre um Corpo de Galois de dimensão 2

In [26]:
Z2 = GF(2)

In [27]:
G = matrix(Z2, [[1,1,0,1,0,0,0],
                [0,1,1,0,1,1,0],
                [1,0,0,1,1,0,1]])
G

[1 1 0 1 0 0 0]
[0 1 1 0 1 1 0]
[1 0 0 1 1 0 1]

In [28]:
G.rank()

3

##### Construção de um código linear com base na matriz geradora G e verificação dos parâmetros: 

In [29]:
C = LinearCode(G)

In [30]:
C

[7, 3] linear code over GF(2)

In [31]:
d = C.minimum_distance()
d

3

**Neste caso a distância mínima do código é 3, ou seja, é um código que deteta até 2 erros e corretor de 1 erro.**

##### Seleção de uma mensagem que é palavra código (combinação linear das linhas de G): 

In [32]:
mens_limpa = vector(Z2, [1,1,1,1,0,1,1])
mens_limpa in C

True

##### Introdução de um erro no segundo bit da mensagem:

In [33]:
mens_err = mens_limpa + vector([0,1,0,0,0,0,0])
mens_err, mens_err in C

((1, 0, 1, 1, 0, 1, 1), False)

##### Matriz de paridade H:

In [34]:
H = C.parity_check_matrix()
H

[1 0 0 1 0 0 0]
[0 1 0 1 0 1 1]
[0 0 1 0 0 1 0]
[0 0 0 0 1 1 1]

##### Confirmação, usando a matriz de paridade, de que a mensagem original (c) é palavra código (H*c = 0):

In [35]:
H*(matrix(mens_limpa).transpose()) #Hc = 0

[0]
[0]
[0]
[0]

###### Confirmação, usando a matriz de paridade, de que a mensagem com o erro introduzido (r) não é palavra código (H*c != 0):

In [36]:
sindrome = H*(matrix(mens_err).transpose()) #Hr != 0
sindrome

[0]
[1]
[0]
[0]

### Correção do erro usando o síndrome obtido anteriormente e matriz identidade de ordem 7:

In [37]:
I7 = identity_matrix(7)
I7

[1 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 1 0 0 0 0]
[0 0 0 1 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 0 1 0]
[0 0 0 0 0 0 1]

##### Ciclo que multiplica a matriz de paridade H pelas diversas coluna de I7. O objetivo é obter o síndrome e saber qual o bit que está errado:

In [38]:
[(H*matrix(col).transpose(), f"Bit {i+1}") for i,col in enumerate(I7.columns())]

[(
[1]         
[0]         
[0]         
[0], 'Bit 1'
),
 (
[0]         
[1]         
[0]         
[0], 'Bit 2'
),
 (
[0]         
[0]         
[1]         
[0], 'Bit 3'
),
 (
[1]         
[1]         
[0]         
[0], 'Bit 4'
),
 (
[0]         
[0]         
[0]         
[1], 'Bit 5'
),
 (
[0]         
[1]         
[1]         
[1], 'Bit 6'
),
 (
[0]         
[1]         
[0]         
[1], 'Bit 7'
)]

**Podemos observar que é segundo bit que está errado como era esperado, visto que a segunda coluna da matriz de paridade H (ou resultado da multiplicação da matriz de paridade H pela segunda coluna da matriz identidade) deu o resultado do síndrome (H*mensagem_errada)**

In [39]:
mens_err

(1, 0, 1, 1, 0, 1, 1)

##### Correção do erro somando à mensagem errada o vetor e2 composto por zeros, com um 1 apenas na segunda posição.

In [40]:
mens_corrigida = mens_err + vector([0,1,0,0,0,0,0])

##### Confirmação de que a mensagem corrigida é palavra código e que é igual à mensagem original:

In [41]:
mens_corrigida, mens_corrigida in C, mens_corrigida == mens_limpa

((1, 1, 1, 1, 0, 1, 1), True, True)

### No mesmo código e com a mesma mensagem original vamos agora repetir o processo mas colocar um erro numa outra posição, neste caso na última.

In [42]:
mens_err_2 = mens_limpa + vector([0,0,0,0,0,0,1])
mens_err_2

(1, 1, 1, 1, 0, 1, 0)

In [43]:
mens_err_2 in C

False

In [44]:
sindrome_2 = H*(matrix(mens_err_2).transpose())
sindrome_2

[0]
[1]
[0]
[1]

In [45]:
H

[1 0 0 1 0 0 0]
[0 1 0 1 0 1 1]
[0 0 1 0 0 1 0]
[0 0 0 0 1 1 1]

In [46]:
[(H*matrix(col).transpose(), f"Bit {i+1}") for i,col in enumerate(I7.columns())]

[(
[1]         
[0]         
[0]         
[0], 'Bit 1'
),
 (
[0]         
[1]         
[0]         
[0], 'Bit 2'
),
 (
[0]         
[0]         
[1]         
[0], 'Bit 3'
),
 (
[1]         
[1]         
[0]         
[0], 'Bit 4'
),
 (
[0]         
[0]         
[0]         
[1], 'Bit 5'
),
 (
[0]         
[1]         
[1]         
[1], 'Bit 6'
),
 (
[0]         
[1]         
[0]         
[1], 'Bit 7'
)]

**Podemos ver que, desta vez, a última coluna de H é que iguala o novo síndrome. Assim, o último bit deve ser o bit a ser corrigido.**

In [47]:
mens_corr_2 = mens_err_2 + vector([0,0,0,0,0,0,1])
mens_corr_2

(1, 1, 1, 1, 0, 1, 1)

In [48]:
mens_corr_2 in C, mens_corr_2 == mens_limpa

(True, True)